<a href="https://colab.research.google.com/github/Smartsie/NLP-Scholars/blob/master/ScienceDirectSearch_2_Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
#from textblob import TextBlob
#from google.colab import drive
#drive.mount('/content/drive')

2020-10-04 21:42:20,504 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\Thierry\AppData\Local\Temp\tika-server.jar.
2020-10-04 21:42:52,699 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to C:\Users\Thierry\AppData\Local\Temp\tika-server.jar.md5.
2020-10-04 21:42:53,407 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [26]:
# If we want to analyze a PDF
from tika import parser
parsedPDF = parser.from_file("37.pdf")
text=parsedPDF["content"]
text=text.replace('\n','')
text=text.replace('-','')

In [33]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import itertools
import collections
import nltk
from nltk import bigrams
from nltk.corpus import stopwords
import re

import warnings
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
#import plotly.express as px

warnings.filterwarnings("ignore")

#sns.set(font_scale=1.5)
#sns.set_style("whitegrid")

In [34]:
#Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thierry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Run Query separately

In [ ]:
df=pd.read_csv('hybrid AND control AND powertrain AND filterdataset.csv')
df=df.dropna(axis=0)

FileNotFoundError: ignored

In [ ]:
df.head()

,Unnamed: 0,@_fa,dc:identifier,dc:title,link,load-date,openaccess,pii,prism:coverDate,prism:doi,prism:publicationName,prism:url,Relevant,Abstract
0,0,True,DOI:10.1016/j.energy.2020.118286,Comparison of four-wheel-drive hybrid powertra...,{'self': 'https://api.elsevier.com/content/art...,2020-07-23T00:00:00.000Z,False,S0360544220313931,2020-10-15,10.1016/j.energy.2020.118286,Energy,https://api.elsevier.com/content/article/pii/S...,2,\n The stringent fuel economy...
1,1,True,DOI:10.1016/j.jclepro.2020.120711,"Representation, generation, and optimization m...",{'self': 'https://api.elsevier.com/content/art...,2020-02-26T00:00:00.000Z,False,S0959652620307587,2020-05-20,10.1016/j.jclepro.2020.120711,Journal of Cleaner Production,https://api.elsevier.com/content/article/pii/S...,2,\n Since the first commercial...
2,2,True,DOI:10.1016/j.apenergy.2020.114873,Optimal control of power-split hybrid electric...,{'self': 'https://api.elsevier.com/content/art...,2020-03-30T00:00:00.000Z,False,S0306261920303858,2020-05-15,10.1016/j.apenergy.2020.114873,Applied Energy,https://api.elsevier.com/content/article/pii/S...,3,\n This paper presents a real...
3,3,True,DOI:10.1016/j.apenergy.2020.114553,A survey of powertrain configuration studies o...,{'self': 'https://api.elsevier.com/content/art...,2020-02-15T00:00:00.000Z,False,S0306261920300659,2020-03-15,10.1016/j.apenergy.2020.114553,Applied Energy,https://api.elsevier.com/content/article/pii/S...,2,"\n Global warming, air pollut..."
4,4,True,DOI:10.1016/j.mechmachtheory.2019.103731,Synthesis and analysis method for powertrain c...,{'self': 'https://api.elsevier.com/content/art...,2019-12-20T00:00:00.000Z,False,S0094114X19323377,2020-04-30,10.1016/j.mechmachtheory.2019.103731,Mechanism and Machine Theory,https://api.elsevier.com/content/article/pii/S...,2,\n Existing hybrid powertrain...


In [ ]:
df.columns

Index(['Unnamed: 0', '@_fa', 'dc:identifier', 'dc:title', 'link', 'load-date',
       'openaccess', 'pii', 'prism:coverDate', 'prism:doi',
       'prism:publicationName', 'prism:url', 'Relevant', 'Abstract'],
      dtype='object')

Data is a tabular data with journal name, link to the publication, date of publication, author and abstract

In [35]:
# Define stemmer é lemmatizer
porter=PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [36]:
def stemSentence(sentence,lem=False):
    token_words=word_tokenize(sentence.lower())
    stem_sentence=[]
    lem_sentence=[]
    for word in token_words:
        if word not in stop_words:
            if lem==False:
                stem_sentence.append(porter.stem(word)+' ')
            else:
                stem_sentence.append(wordnet_lemmatizer.lemmatize(word)+' ')
    return "".join(stem_sentence)

![image.png](attachment:image.png)

In [ ]:
# Strip the 58 first blank characters and remove annoying characters coming from ScienceDirect: @, \u202f, \n
#start=59 
start=30# actualy it depends on the source

df["Clean text"]=df["Abstract"].str.slice(start=59).replace('@','').replace('\u202f','').replace('\n','')
# Prepare container for Lemmetized or Stemmed data
df["Lem data"]="-"

In [ ]:
# Apply Stemmer to the dataframe
# save it under this name:
filename='Hybrid PT Control.csv'
for i in range(0,len(df)):
    df["Lem data"].iloc[i]=stemSentence(df['Clean text'].iloc[i],True)
    if i%100==0:
        df.to_csv(filename)
        print(i)
df.to_csv(filename)

0
100
200
300
400
500
600
700
800
900


In [ ]:
df.to_csv(filename)

In [ ]:
df.head()

,Unnamed: 0,@_fa,dc:identifier,dc:title,link,load-date,openaccess,pii,prism:coverDate,prism:doi,prism:publicationName,prism:url,Relevant,Abstract,Clean text,Lem data
0,0,True,DOI:10.1016/j.ifacol.2019.12.624,PQ Theory-Based Control of Single-Stage V2G Th...,{'self': 'https://api.elsevier.com/content/art...,2020-01-15T00:00:00.000Z,False,S2405896319325674,2019-12-31,10.1016/j.ifacol.2019.12.624,IFAC-PapersOnLine,https://api.elsevier.com/content/article/pii/S...,1,\n This work dealt with a mod...,theory-based control of a vehicle-to-grid (V2...,theory-based control vehicle-to-grid ( v2g ) t...
1,1,True,DOI:10.1016/j.apenergy.2019.114262,Two-stage stochastic sizing and packetized ene...,{'self': 'https://api.elsevier.com/content/art...,2019-12-11T00:00:00.000Z,False,S030626191931949X,2020-02-15,10.1016/j.apenergy.2019.114262,Applied Energy,https://api.elsevier.com/content/article/pii/S...,1,\n The expected deployment of...,ric vehicles (BEVs) strongly depends on the de...,ric vehicle ( bevs ) strongly depends developm...
2,2,True,DOI:10.1016/j.ifacol.2019.12.738,BEV Remaining Range Estimation Based on Modern...,{'self': 'https://api.elsevier.com/content/art...,2020-01-15T00:00:00.000Z,False,S2405896319326874,2019-12-31,10.1016/j.ifacol.2019.12.738,IFAC-PapersOnLine,https://api.elsevier.com/content/article/pii/S...,1,\n The aim of this paper is t...,nature of algorithms for calculation of the ra...,nature algorithm calculation range battery ele...
3,3,True,DOI:10.1016/j.apenergy.2018.04.073,Effects of charging battery electric vehicles ...,{'self': 'https://api.elsevier.com/content/art...,2018-05-11T00:00:00.000Z,False,S0306261918306330,2018-08-15,10.1016/j.apenergy.2018.04.073,Applied Energy,https://api.elsevier.com/content/article/pii/S...,2,\n Integration of battery ele...,(BEV) as load could have an impact on the sta...,( bev ) load could impact stability local grid...
4,4,True,DOI:10.1016/j.trc.2019.12.021,Multi-day scenario analysis for battery electr...,{'self': 'https://api.elsevier.com/content/art...,2020-01-09T00:00:00.000Z,False,S0968090X18312117,2020-02-29,10.1016/j.trc.2019.12.021,Transportation Research Part C: Emerging Techn...,https://api.elsevier.com/content/article/pii/S...,1,\n Multi-day activity-travel ...,create potential vehicle usage profiles that c...,create potential vehicle usage profile contain...


In [37]:
stemSentence(text,True)

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\Thierry/nltk_data'
    - 'C:\\Users\\Thierry\\Anaconda3\\envs\\tf\\nltk_data'
    - 'C:\\Users\\Thierry\\Anaconda3\\envs\\tf\\share\\nltk_data'
    - 'C:\\Users\\Thierry\\Anaconda3\\envs\\tf\\lib\\nltk_data'
    - 'C:\\Users\\Thierry\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
